# Polar stereographic projection



In [1]:
import sys, os
import numpy as np


# get current directory\n
path = os.getcwd()
# get parent directory
parent_directory = "/home/llu"
#T_Barrier directory
T_Barrier_directory = parent_directory+"/Programs/TBarrier-main/TBarrier/2D/"

# add convert folder to current working path
sys.path.append(T_Barrier_directory+"data/AVISO")

## Read data

In [2]:
import netCDF4 as nc

# Open the NetCDF file
file_path = parent_directory+'/Data/NeXtSIM/OPA-neXtSIM_CREG025_ILBOXE140_2010_ice.nc'  
dataset = nc.Dataset(file_path, mode='r')

coordinates_path =  parent_directory+'/Data/NeXtSIM/OPA-neXtSIM_CREG025_ILBOXE140_2010_gridU_oce.nc'  
coordinates = nc.Dataset(coordinates_path, mode='r')

metadata_path =  parent_directory+'/Data/NeXtSIM/mesh_mask_CREG025_3.6_NoMed.nc'  
metadata = nc.Dataset(metadata_path, mode='r')

# Access specific variables
time = dataset.variables['time'][:]  
siu = dataset.variables['siu'][:]  
siv = dataset.variables['siv'][:]  

# Access coordinates
latitude = coordinates.variables['nav_lat'][:]  
longitude = coordinates.variables['nav_lon'][:]

# Access metadata
e1u = metadata.variables['e1u'][:]  
e2v = metadata.variables['e2v'][:]

# Close the dataset when done
dataset.close()
coordinates.close()
metadata.close()


In [3]:
latitude.shape

(603, 528)

In [4]:
## Cut the length of the timeseries to 20 for optimisation during developing

In [5]:
siu = siu[:200, :, :]
siv = siv[:200, :, :]
time = time[200:1]

In [6]:
## Change of units

Note that the velocities $siu$ and $siv$ are given in $m/s$. Since we advect the trajectories in latitude and longitude, we need to transform the units to $deg/s$. \
To do so we use the function convert_meters_per_second_to_deg_per_day$(X, Y, U\_ms, V\_ms)$ from the TBarrier package.

In [7]:
# Import convert_meters_per_second_to_deg_per_day function for unsteady flow field
from ipynb.fs.defs.convert_meters_per_second_to_deg_per_day import convert_meters_per_second_to_deg_per_day
import numpy as np 

In [8]:
#Transpose the velocity matrix to match the dimensions required by the function
siu_t = np.transpose(siu, (1, 2, 0))
siv_t = np.transpose(siv, (1, 2, 0))

In [9]:
siu.shape

(200, 603, 528)

In [10]:
siu_deg_day, siv_deg_day = convert_meters_per_second_to_deg_per_day(longitude, latitude , siu_t, siv_t)

KeyboardInterrupt: 

In [12]:
#Go to original dimensions (t,x,y)
siu_deg_day = np.transpose(siu_deg_day, (2, 0, 1))
siv_deg_day = np.transpose(siv_deg_day, (2, 0, 1))

## Define the polar stereographic projection

In [ ]:
import pyproj

# Define the Polar Stereographic projection using the provided proj4 string
polar_stereographic_proj = (
    '+proj=stere +a=6378273 +b=6356889.5 +lat_0=90 +lat_ts=60 +lon_0=-45 '
    '+x_0=1 +y_0=1'
)

transformer_inv = pyproj.Transformer.from_proj(
    proj_from = polar_stereographic_proj,  # Target Polar Stereographic projection
    proj_to = 'epsg:4326',  # Source coordinate system (WGS84 Lat/Lon)
    always_xy=True
)

transformer = pyproj.Transformer.from_proj(
    proj_from = 'epsg:4326',  # Source coordinate system (WGS84 Lat/Lon)
    proj_to = polar_stereographic_proj,  # Target Polar Stereographic projection
    always_xy=True
)

def polar_stereo(a, b, inverse=False):
    """
    Converts longitude (a) and latitude (b) to x (i), y (j) coordinates using the Polar Stereographic projection by default. If inverse=True
    then it converts x (a) and y (b) to longitude (i) and latitude (j)

    Parameters:
    a (float): Latitude in degrees.
    b (float): Longitude in degrees.

    Returns:
    (float, float): i, j coordinates in the Polar Stereographic projection.
    """

    
    # Create a pyproj Transformer object for the given projection
    if inverse:
        # Transformation
        i, j = transformer_inv.transform(a, b)
    else:
        # Transformation
        i, j = transformer.transform(a, b)
    
    return i, j


# Transforme lat lon data to x y 

In [50]:
x, y = polar_stereo(longitude,latitude, inverse=False)

Check that the inverse function is well written

In [51]:
lon, lat = polar_stereo(x,y,inverse=True)

# Transform velocities of the x,y components to velocities of the longitude, latitude components 

The function "Jacobian" computes the numerical Jacobian matrix of a two-dimensional function $f: \mathbb{R}^2 \rightarrow \mathbb{R}^2$. If inverse=True, then it computes the numerical Jacobian of $f^{-1}$
$$J=\begin{pmatrix}
\frac{\partial f_x}{\partial x} & \frac{\partial f_x}{\partial y} \\
\frac{\partial f_y}{\partial x} & \frac{\partial f_y}{\partial y} 
\end{pmatrix}$$



In [10]:
def Jacobian(f, x, y, h=1e-5, inverse=False):
    """
    Compute the numerical Jacobian of a 2D vector function f at point (x, y).

    Parameters:
    f: function
       Function that takes two arguments (x, y) and returns a tuple or list with two elements (f1, f2).
    x: float
       x-coordinate where the Jacobian is to be evaluated.
    y: float
       y-coordinate where the Jacobian is to be evaluated.
    h: float, optional
       Step size for finite differences (default is 1e-5).
       
    Returns:
    J: 2x2 numpy array
       Jacobian matrix evaluated at (x, y).
    """
    # Initialize a 2x2 matrix for the Jacobian
    J = np.zeros((2, 2))
    
    # Partial derivatives with respect to x
    f_xplus = f(x + h, y, inverse)  # f(x+h, y)
    f_xminus = f(x - h, y, inverse)  # f(x-h, y)
    J[:, 0] = (np.array(f_xplus) - np.array(f_xminus)) / (2 * h)
    
    # Partial derivatives with respect to y
    f_yplus = f(x, y + h, inverse)  # f(x, y+h)
    f_yminus = f(x, y - h, inverse)  # f(x, y-h)
    J[:, 1] = (np.array(f_yplus) - np.array(f_yminus)) / (2 * h)
    
    return J


Here we use the Jacobian to transform the velocities of the x, and y components to velocities of the longitude and latitude components. For that, we define the function "vel_trans" which implements the matrix multiplication
$$
\begin{bmatrix}
v_{\text{lat}} \\
v_{\text{lon}}
\end{bmatrix}
= J \cdot
\begin{bmatrix}
v_x \\
v_y
\end{bmatrix}$$
Resulting Equations
$v_{\text{lat}} = \frac{\partial \text{lat}}{\partial x} \cdot v_x + \frac{\partial \text{lat}}{\partial y} \cdot v_y$
$v_{\text{lon}} = \frac{\partial \text{lon}}{\partial x} \cdot v_x + \frac{\partial \text{lon}}{\partial y} \cdot v_y$

In [53]:
import sys
from timeit import default_timer as timer

def vel_transf_loop(J, f, X, Y, siu, siv, h=1e-5, inverse=False):
    
    siu_transf= siu.copy()
    siv_transf= siv.copy()
    if np.array_equal(siu.mask, siv.mask)==False:
        print("siu and siv have different masks!")
        sys.exit(1)
    start = timer()
    for i in range(siu.shape[0]):
        print(i)
        for j in range(siu.shape[1]):
            for k in range(siu.shape[2]):
                if siu.mask[i,j,k]== False:
                    siu_transf[i,j,k] = Jacobian(polar_stereo,x[j,k],y[j,k],inverse=inverse)[0,0]#np.array([siu_deg_day[i,j,k],siv_deg_day[i,j,k]]) #np.dot(Jacobian(polar_stereo,x[j,k],y[j,k],inverse=inverse),np.array([siu_deg_day[i,j,k],siv_deg_day[i,j,k]]))
            print(j)
            end = timer()
            print(end - start)
            start = timer()
    return siu_transf, siv_transf

In [28]:
def vel_transf_loop2(J, f, X, Y, siu, siv, h=1e-5, inverse=False):
    
    siu_transf= siu.copy()
    siv_transf= siv.copy()
    if np.array_equal(siu.mask, siv.mask)==False:
        print("siu and siv have different masks!")
        sys.exit(1)
    start = timer()
        
    for j in range(siu.shape[1]):
        print(j)
        for k in range(siu.shape[2]):
            J = Jacobian(polar_stereo,x[j,k],y[j,k],inverse=inverse)
            for i in range(siu.shape[0]):
                if siu.mask[i,j,k]== False:
                    siu_transf[i,j,k], siv_transf[i,j,k] = np.dot(J,np.array([siu_deg_day[i,j,k],siv_deg_day[i,j,k]]))
        end = timer()
        print(end - start)
        start = timer()
    return siu_transf, siv_transf

In [ ]:
siu_pol_deg_day, siv_pol_deg_day = vel_transf_loop(Jacobian, polar_stereo, x, y, siu_deg_day, siv_deg_day, inverse=True)

In [ ]:
siu_pol_deg_day2, siv_pol_deg_day2 = vel_transf_loop2(Jacobian, polar_stereo, x, y, siu_deg_day, siv_deg_day, inverse=True)

Non masked example:
$$i=5 \hspace{1cm}
j=416  \hspace{1cm}
k=286$$

## Relocation of the Npole
In this section, we use:\
The rotations around the cartesian axis
$$R_x(\theta) = \begin{bmatrix} 1 & 0 & 0\\ 0 & \cos\left(\theta\right) & -\sin\left(\theta\right) \\ 0 & \sin\left(\theta\right)& cos\left(\theta \right)\end{bmatrix} \hspace{1cm}
R_y(\theta) = \begin{bmatrix} \cos\left(\theta\right) & 0 & \sin\left(\theta\right)\\ 0 & 1 & 0 \\ -\sin\left(\theta\right) & 0 & cos\left(\theta \right)\end{bmatrix} \hspace{1cm}
R_z(\theta) = \begin{bmatrix} \cos\left(\theta\right) & -\sin\left(\theta\right) & 0\\ \sin\left(\theta\right) & \cos\left(\theta\right) & 0 \\ 0 & 0 & 1\end{bmatrix}$$


Transformation from cartesian to spherical and viceversa
$$\begin{bmatrix} x \\ y \\ z \end{bmatrix} = r \begin{bmatrix} \sin(\theta) \cos(\varphi) \\ \sin(\theta) \sin(\varphi) \\ \cos(\theta) \end{bmatrix}$$


Define the positive rotation matrices around the cartesian axis. This means that the rotation is according to the right hand rule. Note that they are defined for the cartesian coordinates

In [19]:
import math
def Rx(psi):
    return np.matrix([[ 1, 0           , 0           ],
                       [ 0, m.cos(psi), -m.sin(psi)],
                       [ 0, m.sin(psi), m.cos(psi)]])
  
def Ry(psi):
    return np.matrix([[ m.cos(psi), 0, m.sin(psi)],
                       [ 0          , 1, 0          ],
                       [-m.sin(psi), 0, m.cos(psi)]])
  
def Rz(psi):
    return np.matrix([[ m.cos(psi), -m.sin(psi), 0 ],
                       [ m.sin(psi), m.cos(psi) , 0 ],
                       [ 0          , 0           , 1 ]])

Define the transformation from spherical to cartiesian coordinates and viceversa

In [20]:
def spherical_to_cartesian(r, theta, phi):
    """
    Converts spherical coordinates (r, theta, phi) to Cartesian coordinates (x, y, z).
    
    Parameters:
        r (float): Radial distance
        theta (float): Polar angle in radians (0 <= theta <= pi)
        phi (float): Azimuthal angle in radians (0 <= phi < 2*pi)
        
    Returns:
        tuple: Cartesian coordinates (x, y, z)
    """
    x = r * math.sin(theta) * math.cos(phi)
    y = r * math.sin(theta) * math.sin(phi)
    z = r * math.cos(theta)
    
    return x, y, z

def cartesian_to_spherical(x, y, z):
    """
    Converts Cartesian coordinates (x, y, z) to spherical coordinates (r, theta, phi).
    
    Parameters:
        x (float): X-coordinate
        y (float): Y-coordinate
        z (float): Z-coordinate
        
    Returns:
        tuple: Spherical coordinates (r, theta, phi)
    """
    # Radial distance
    #r = math.sqrt(x**2 + y**2 + z**2)
    # Radial distance in the surface
    earthRadius = 6371*(10**3)
    r = earthRadius
    
    # Polar angle (theta)
    theta = math.acos(z / r) if r != 0 else 0  # Handle zero-division at the origin
    
    # Azimuthal angle (phi)
    phi = math.atan2(y, x)  # atan2 handles the correct quadrant for (y, x)
    
    return r, theta, phi

Let's define the complete transformation for $R_y(90^0)$:

In [26]:
def polar_rotation(long,lat,Rx,psix):
    #psix in radiants
    # Radius of the earth
    earthRadius = 6371*(10**3)
    #r = np.full(latitude.shape, earthRadius)
    x,y,z = spherical_to_cartesian(earthRadius,degrees_to_radiants(lat),degrees_to_radiants(long))
    vect=np.dot(Rx(psix), np.array([[x],[y],[z]]))
    return cartesian_to_spherical(vect[0],vect[1],vect[2])
    

# Unit transformations

In [ ]:
eq_axis = 6378.1*(10**3) #m
eq_rad = 2*math.pi*eq_axis #m; circumference along the equator
polar_axis = 6356.8*(10**3) #m
polar_rad = 2*math.pi*polar_axis #m; circumference along the NP and SP
 
from math import cos, pi, radians

#m to lat_rot

def m_to_lat_r(dist_m):
    return (dist_m*360/eq_rad)

def m_to_lon_r(dist_m,lat_rad):
     return cos(radians(lat_rad))*dist_m* 360/polar_rad 


#m_to_lat_r(111000)
m_to_lon_r(polar_rad,0)



In [75]:
def m_to_deg(Vy, Vx, latitude):

    eq_radius = 6378.1*(10**3) #m 
    eq_circ = 2*math.pi*eq_radius #m; circumference along the equator
    polar_radius = 6356.8*(10**3) #m
    polar_circ = 2*math.pi*polar_radius #m; circumference along the NP and SP

    Vlat = Vy*360/eq_circ
    Vlon = np.multiply(np.cos(np.radians(latitude))[:,:,np.newaxis],Vx)*360/polar_circ
    
    return Vlat, Vlon

In [82]:
siv_deg, siu_deg = m_to_deg(siv,siu,latitude)

In [ ]:
siu_deg[424,320,0]

In [ ]:
m_to_lon_r(siu[424,320,0],latitude[424,320])

In [ ]:
siv_deg[424,320,0]

In [ ]:
m_to_lat_r(siv[424,320,0])